In [110]:
from pyqubo import Array, Constraint, Placeholder
import time

In [115]:
n_city = 2

t0 = time.time()
x = Array.create('c', (n_city, n_city), 'BINARY')

# Constraint not to visit more than two cities at the same time.
time_const = 0.0
for i in range(n_city):
    # If you wrap the hamiltonian by Const(...), this part is recognized as constraint
    time_const += Constraint((sum(x[i, j] for j in range(n_city)) - 1)**2, label="time{}".format(i))

# Constraint not to visit the same city more than twice.
city_const = 0.0
for j in range(n_city):
    city_const += Constraint((sum(x[i, j] for i in range(n_city)) - 1)**2, label="city{}".format(j))

# distance of route
distance = 0.0
for i in range(n_city):
    for j in range(n_city):
        for k in range(n_city):
            # we set the constant distance
            d_ij = abs(i-j)
            distance += d_ij * x[k, i] * (x[(k+1)%n_city, j] + x[(k-1)%n_city, j])

# Construct hamiltonian
A = Placeholder("A")
H = distance + A * (time_const + city_const)

# Compile model
t1 = time.time()
model = H.compile()
qubo, offset = model.to_qubo(index_label=True, feed_dict={"A": 2.0})
l,q,o = model.to_ising(index_label=True, feed_dict={"A": 2.0})
t2 = time.time()

In [116]:
sorted(l.items(), key=lambda x:x[0])

[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0)]

In [117]:
import numpy as np
a = np.zeros((n_city**2, n_city**2))

for key,value in q.items():
    a[key] = value
a

array([[0., 1., 1., 1.],
       [0., 0., 1., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.]])

In [118]:
o

6.0